In [24]:
from rdkit import Chem, rdBase
from rdkit.Chem import Draw,rdDepictor
from rdkit.Chem import AllChem, Descriptors, DataStructs
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from datetime import datetime
# import math
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# import os
# import pyvis
# from pyvis.network import Network
# import inspect
import psycopg2
from collections import defaultdict
import collections
import sys
sys.path.append('/mnt/raid1/Python_projects')
sys.path.append('/mnt/raid1/Python_projects/pkg_mod')

##### Custom Modules #####
import chemistry_psql as cp
import fingerprinter as fpt


##### Tanimoto Backfill #####

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres") 

cur = db_chem.cursor() 
sql = 'SELECT structure_id, smiles FROM public.structures ORDER BY structure_id' 
cur.execute(sql) 
mollist = cur.fetchall()
#print(mollist)

cur = db_chem.cursor() 
sql = 'select s.structure_id \
from public.structures s \
where s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.tanimoto fp WHERE f.structure_id = fp.target)) \
AND s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.tanimoto fp WHERE f.structure_id = fp.source)) \
ORDER BY s.structure_id' 
cur.execute(sql) 
m = cur.fetchall()
#print(m)

for n in m[1:]:    
    tm = n[0]
    #print(n[0])
    cur = db_chem.cursor() 
    sql = 'SELECT structure_id, smiles FROM public.structures WHERE structure_id <= '+ str(tm) +' ORDER BY structure_id' 
    cur.execute(sql) 
    mollist = cur.fetchall()
    #print(mollist)
    maxtan = 0
    ml = len(mollist)
    #print(tm, ml)
    #tp = mollist[0][0]
    cur = db_chem.cursor()
    sql = 'INSERT INTO public.tanimoto (source, target, tanimoto) VALUES (%s,%s,%s);'
    m2 = Chem.MolFromSmiles(mollist[ml-1][1])
    fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
    for j in range (0, ml-1):
        m1 = Chem.MolFromSmiles(mollist[j][1])
        fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
        tan = DataStructs.TanimotoSimilarity(fp1,fp2)
        if tan > maxtan:
            #print(mollist[j][0],mollist[ml-1][0], tan, maxtan)
            maxtan = tan
            ms = mollist[j][0]
            mt = mollist[ml-1][0]
    if maxtan>0:
        print('Max Tani : ',ms, mt, maxtan)
        cur.execute(sql, (ms,mt, maxtan))
    else:
        print('Tani failed : ', tm)

db_chem.commit()
db_chem.close()

11 2
Max Tani :  10 11 0.3333333333333333
14 4
Max Tani :  13 14 0.2
15 5
Max Tani :  13 15 0.35714285714285715
17 7
Tani failed :  17
18 8
Max Tani :  17 18 0.3333333333333333
19 9
Max Tani :  17 19 0.3103448275862069
20 10
Max Tani :  19 20 0.3793103448275862
21 11
Max Tani :  20 21 0.42857142857142855
22 12
Max Tani :  21 22 0.4
23 13
Max Tani :  22 23 0.3939393939393939
24 14
Max Tani :  16 24 0.375
35 16
Max Tani :  15 35 0.375
36 17
Max Tani :  16 36 0.375
37 18
Max Tani :  24 37 0.2727272727272727
39 20
Max Tani :  19 39 0.17142857142857143
40 21
Max Tani :  39 40 0.23232323232323232
41 22
Max Tani :  39 41 0.24489795918367346
49 30
Max Tani :  46 49 0.12222222222222222
51 32
Max Tani :  50 51 0.4966887417218543
63 40
Max Tani :  40 63 0.19327731092436976
64 41
Max Tani :  47 64 0.25735294117647056
68 45
Max Tani :  17 68 0.125
69 46
Max Tani :  68 69 0.2857142857142857
70 47
Max Tani :  17 70 0.2222222222222222
71 48
Max Tani :  68 71 0.3333333333333333
72 49
Max Tani :  69 72 

KeyboardInterrupt: 